<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<center><font size="8pt">Classification with Support Vector Machine</font></center>

# Introduction 

This `R` tutorial deals with Support Vector Machine (SVM) for classification. The aim is to illustrate the importance of the kernel choice and the tuning of the parameters by cross validation on different simulated data. 

It uses the `svm()` function of the `e1071` package. 

## About the `svm()` function

**Q:** Load the `e1071` package. What kernels does it contain?

## Factor simulation

In the following, the factors $(X_i)_{1\leq i\leq n}$ are the same in all cases and are uniformly distributed on $[-1,1]\times [0,1]$. The difference between the different cases lies in the boundary form which classifies the points. We simulate a training set with `n=200` observations, and a test set with `ntest=100` points. 

In [ ]:
# Training set:
n <- 200
x1 <- runif(n, min=-1, max=1)
x2 <- runif(n, min=0, max=1)
t <- seq(-1, 1, length=100)

# Test set:
ntest <- 100
x1test <- runif(ntest, min=-1, max=1)
x2test <- runif(ntest, min=0, max=1)

# Linear boundary

First, we assume that the points are linearly separated. 

## Data simulation

In [ ]:
boundFunLin <- function(x1){
  x1 + 0.5
}

# Training set
gLin <- as.factor(x2 > boundFunLin(x1))
dataTrainLin <- data.frame(x1 = x1, x2 = x2, g = gLin)
plot(x2, x1, col=as.numeric(gLin))
lines(boundFunLin(t), t, col="blue", lty = "dotted")

# Test set
gLintest <- as.factor(x2test > boundFunLin(x1test))
dataTestLin <- data.frame(x1 = x1test, x2 = x2test, g = gLintest)
points(x2test, x1test, col=as.numeric(gLintest),pch=10)


## Linear SVM

We first use a linear kernel. 

In [ ]:
svm.model <- svm(g ~ ., data = dataTrainLin, type = "C", kernel = "lin", cross = 5)
plot(svm.model, data = dataTrainLin)

In [ ]:
summary(svm.model)

**Q:** How many support vectors are there? How are they represented on the plot? 
What is their minimal number? Can you change the parameters, such that this minimal number is achieved?

**Q:** What are the `Single Accuracies` and the `Total Accuracy` in the `summary`? Why should we do some cross validation, even though there are no parameters to calibrate in the linear model?

**Q:** Verify that the fitted values are equal to the sign of the decision values. 

In [ ]:
unique((svm.model$decision.values<0) == (svm.model$fitted))

Note that it is possible to get the normalized support vectors. 

In [ ]:
par(mfrow=c(1,2))
plot(x1~x2,data=svm.model$SV)
plot(x1~x2,data=dataTrainLin[svm.model$index,])

## Test set prediction

Let us now compute the contingency table and accuracy rate of `svm.model` on the test set. 

In [ ]:
# Generalization error
pred.lin <- predict(svm.model,dataTestLin)
table(pred.lin,dataTestLin$g) 
paste("Generalization error: ",100*(1-sum(diag(table(pred.lin,dataTestLin$g)))/ntest),"%",sep="")

In [ ]:
plot(svm.model,dataTestLin)

**Q:** Comment. 

# Cubic boundary

In this section, we assume that the points are separated by a cubic function (polynomial with degree 3). 

## Data simulation

In [ ]:
boundFunCub <- function(u){
  2*u^2 - 10*u^3 + 0.5
}

# Training set
gCub <- (x2 > boundFunCub(x1))
gCub <- as.factor(gCub)
dataTrainCub <- data.frame(x1 = x1, x2 = x2, g = gCub)
plot(x2, x1, col=as.numeric(gCub))
lines(boundFunCub(t), t, col="blue", lty = "dotted")

# Test set
gCubtest <- as.factor(x2test > boundFunCub(x1test))
dataTestCub <- data.frame(x1 = x1test, x2 = x2test, g = gCubtest)
points(x2test, x1test, col=as.numeric(gCubtest),pch=10)

## Linear SVM

With a linear kernel, we obtain the following.

In [ ]:
svm.Cub.lin <- svm(g ~ ., data = dataTrainCub, type = "C", kernel = "lin", cross = 5)
plot(svm.Cub.lin, data=dataTrainCub)

In [ ]:
summary(svm.Cub.lin)

**Q:** What is the generalization error here? Comment. 

## Polynomial SVM

Since we know here that the boundary is polynomial with degree 3, we can use a polynomial kernel.

### Polynomial kernel with default parameters

**Q:** What are the parameters of such kernel, and what is their default value?

In [ ]:
svm.Cub.poly <- svm(g ~ ., data = dataTrainCub, type = "C", kernel = "poly", cross = 5)
plot(svm.Cub.poly, data = dataTrainCub)

In [ ]:
summary(svm.Cub.poly)

**Q:** Does this model seem reasonable? What should we do to improve it?

In order to study the training error, we can print the contingency table of the fitted values. 

In [ ]:
table(svm.Cub.poly$fitted,dataTrainCub$g) # Il faudrait sur échantillon test !
paste("Training error: ",round(100*(1-sum(diag(table(svm.Cub.poly$fitted,dataTrainCub$g)))/nrow(dataTrainCub)),1),"%",sep="")

**Q:** Comment.

### Tuned model

Since we know the degree in this example, we only tune the parameter `coef0`. 

In [ ]:
svm.Cub.poly.tune <- tune.svm(g ~ ., data = dataTrainCub, type = "C", kernel = "poly", coef0 = -5:5)
plot(svm.Cub.poly.tune$best.model, data = dataTrainCub)

In [ ]:
# print(svm.Cub.poly.tune)
summary(svm.Cub.poly.tune)

**Q:** What value of `coef0` is obtained by cross validation? What is the generalization error in this case?

We can plot the generalization error (estimated by cross validation) w.r.t. `coef0`. 

In [ ]:
#svm.Cub.poly.tune$best.parameters
plot(svm.Cub.poly.tune)

**Q:** Which parameter minimizes the error?  

## Radial SVM 

### Radial kernel with default parameters

**Q:** What are the parameters of such kernel, and what is their default value?

In [ ]:
svm.Cub.rad <- svm(g ~ ., data = dataTrainCub, type = "C", kernel = "radial", cross = 5)
plot(svm.Cub.rad, data = dataTrainCub) 

In [ ]:
summary(svm.Cub.rad)

**Q:** How much is the generalization error (estimated by cross validation) here? Does the model seem reasonable?

### Tuned model

In [ ]:
svm.Cub.rad.tune <- tune.svm(g ~ ., data = dataTrainCub, type = "C", kernel = "radial", gamma = seq(0.1, 2, by = 0.2))
plot(svm.Cub.rad.tune$best.model, data = dataTrainCub)

In [ ]:
# print(svm.Cub.rad.tune)
summary(svm.Cub.rad.tune)

 We can plot the estimated generalization error w.r.t. `gamma`. 

In [ ]:
plot(svm.Cub.rad.tune)
# plot(error~gamma,data=svm.Cub.rad.tune$performances,type="b")
# svm.Cub.rad.tune$best.parameters

**Q:** Which parameter minimizes the error? What is the generalization error in that case? What is its values for the default parameter? 

## Model comparison

### Validation error (by cross validation)

Comparison of the cross-validation errors:

In [ ]:
paste("Linear kernel:",100-svm.Cub.lin$tot.accuracy,"%",sep=" ")
paste("Default polynomial kernel:",100-svm.Cub.poly$tot.accuracy,"%",sep=" ")
paste("Tuned polynomial kernel:",100*(svm.Cub.poly.tune$best.performance),"%",sep=" ")
paste("Default radial kernel:",100-svm.Cub.rad$tot.accuracy,"%",sep=" ")
paste("Tuned radial kernel:",100*(svm.Cub.rad.tune$best.performance),"%",sep=" ")

**Q:** Which model seems best?

### Test set prediction

In [ ]:
plot(x2test, x1test, col=as.numeric(gCubtest),pch=10)
lines(boundFunCub(t), t, col="blue", lty = "dotted")

In [ ]:
# Linear kernel
pred.Cub.lin <- predict(svm.Cub.lin,dataTestCub)
table(pred.Cub.lin,dataTestCub$g) 
paste("Prediction error: ",round(100*(1-sum(diag(table(pred.Cub.lin,dataTestCub$g)))/ntest),1)," %",sep="")
plot(svm.Cub.lin,dataTestCub)

In [ ]:
# Default polynomial kernel
pred.Cub.poly <- predict(svm.Cub.poly,dataTestCub)
table(pred.Cub.poly,dataTestCub$g) 
paste("Prediction error: ",round(100*(1-sum(diag(table(pred.Cub.poly,dataTestCub$g)))/ntest),1)," %",sep="")
plot(svm.Cub.poly,dataTestCub)

In [ ]:
# Tuned polynomial kernel
pred.Cub.poly.tune <- predict(svm.Cub.poly.tune$best.model,dataTestCub)
table(pred.Cub.poly.tune,dataTestCub$g) 
paste("Prediction error: ",100*(1-sum(diag(table(pred.Cub.poly.tune,dataTestCub$g)))/ntest)," %",sep="")
plot(svm.Cub.poly.tune$best.model,dataTestCub)

In [ ]:
# Default radial kernel
pred.Cub.rad <- predict(svm.Cub.rad,dataTestCub)
table(pred.Cub.rad,dataTestCub$g) 
paste("Prediction error: ",100*(1-sum(diag(table(pred.Cub.rad,dataTestCub$g)))/ntest)," %",sep="")
plot(svm.Cub.rad,dataTestCub)

In [ ]:
# Tuned radial kernel
pred.Cub.rad.tune <- predict(svm.Cub.rad.tune$best.model,dataTestCub)
table(pred.Cub.rad.tune,dataTestCub$g) 
paste("Prediction error: ",round(100*(1-sum(diag(table(pred.Cub.rad.tune,dataTestCub$g)))/ntest),1)," %",sep="")
plot(svm.Cub.rad.tune$best.model,dataTestCub)

In [ ]:
# Prediction accuracies
paste("Linear kernel:",round(100*(1-sum(diag(table(pred.Cub.lin,dataTestCub$g)))/ntest),1),"%",sep=" ")
paste("Default polynomial kernel:",round(100*(1-sum(diag(table(pred.Cub.poly,dataTestCub$g)))/ntest)),"%",sep=" ")
paste("Tuned polynomial kernel:",round(100*(1-sum(diag(table(pred.Cub.poly.tune,dataTestCub$g)))/ntest)),"%",sep=" ")
paste("Default radial kernel:",round(100*(1-sum(diag(table(pred.Cub.rad,dataTestCub$g)))/ntest)),"%",sep=" ")
paste("Tuned radial kernel:",round(100*(1-sum(diag(table(pred.Cub.rad.tune,dataTestCub$g)))/ntest)),"%",sep=" ")

**Q:** Conclude. 

# Absolute value boundary

In this section, we assume that the points are separated by an absolute value function. 

## Data simulation

In [ ]:
boundFunAbs <- function(x1){
  abs(x1)
}

# Training set
gAbs <- (x2 > boundFunAbs(x1))
gAbs <- as.factor(gAbs)
dataTrainAbs <- data.frame(x1 = x1, x2 = x2, g = gAbs)
plot(x2, x1, col=as.numeric(gAbs))
lines(boundFunAbs(t), t, col="blue", lty = "dotted")

# Test set
gAbstest <- as.factor(x2test > boundFunAbs(x1test))
dataTestAbs <- data.frame(x1 = x1test, x2 = x2test, g = gAbstest)
points(x2test, x1test, col=as.numeric(gAbstest),pch=10)

## Radial SVM

### Radial kernel with default parameters

**Q:** Train a first SVM classifier `svm.Abs.rad` based on a radial kernel with default parameters. 

### Radial kernel with `cost=1000`

**Q:** Train a second SVM classifier `svm.Abs.rad.cost1000` based on a radial kernel with default parameters, with `cost=1000`. What can you observe? 
What is the effect of the cost w.r.t. overfitting?

### Tuned radial model

**Q:** Tune a third model `svm.Abs.rad.tune` based on a radial kernel by cross validation. The parameters can be taken in the following grids: 
- `gamma = seq(0.1, 2, by = 0.2)`, 
- `cost = c(1, 25, 50, 75, 100, 150, 200)`. 

**Q:** For which parameters is the generalization error the smallest? What is its value in that case? 

**Q:** Plot the generalization error w.r.t `gamma` and `cost`. What do you observe?

## Test set prediction

**Q:** Compare the three radial models `svm.Abs.rad`, `svm.Abs.rad.tune.cost1000` and `svm.Abs.rad.tune` on the test set (in terms of contingency table and generalization error). Comment. 

# Sine boundary

## Data simulation

In [ ]:
boundFunSin <- function(x1){
  sin(2*pi*x1)
}

# Training set
gSin <- (x2 > boundFunSin(x1))
gSin <- as.factor(gSin)
dataTrainSin <- data.frame(x1 = x1, x2 = x2, g = gSin)
plot(x2, x1, col=as.numeric(gSin))
lines(boundFunSin(t), t, col="blue", lty = "dotted")

# Test set
gSintest <- as.factor(x2test > boundFunSin(x1test))
dataTestSin <- data.frame(x1 = x1test, x2 = x2test, g = gSintest)
points(x2test, x1test, col=as.numeric(gSintest),pch=10)


**Q:** Same questions as for the Absolute value boundary case (with radial kernels). 

## Radial SVM

### Default parameters

### With `cost=1000`

### Tuned model

## Model comparison

### Validation error (by cross validation)

### Test set prediction

# Disk boundary

## Data simulation

In [ ]:
boundFunDisk <- function(x1, x2){
   x1^2 + (x2 - 1/2)^2
}
r <- 0.4

# Training set
gDisk <- as.factor(boundFunDisk(x1, x2) < r^2)
dataTrainDisk <- data.frame(x1 = x1, x2 = x2, g = gDisk)
plot(x2, x1, col=as.numeric(gDisk))
lines(1/2 + r * cos(2*pi*t), r * sin(2*pi*t), col="blue", lty = "dotted")

# Test set
gDisktest <- as.factor(boundFunDisk(x1test, x2test) < r^2)
dataTestDisk <- data.frame(x1 = x1test, x2 = x2test, g = gDisktest)
points(x2test, x1test, col=as.numeric(gDisktest),pch=10)


**Q:** Same questions as for the Absolute value boundary case (with polynomial and radial kernels). 

## Polynomial SVM

### Default parameters

### Tuned model

## Radial SVM 

### Default parameters

### Tuned model

## Model comparison

### Validation error (by cross validation)

### Test set prediction